# 희소 표현(Sparse Representation)
-  벡터의 특정 차원에 단어 혹은 의미를 직접 매핑하는 방식

# 단어의 분산 표현(Distributed Representation)
- 분포 가설(distribution hypothesis) : 유사한 맥락에서 나타나는 단어는 그 의미도 비슷하다.
- 그저 유사한 맥락에 나타난 단어들끼리는 두 단어 벡터 사이의 거리를 가깝게 하고, 그렇지 않은 단어들끼리는 멀어지도록 조금씩 조정해 주는 것.
-  의미가 벡터의 여러 차원에 분산되어 있음.
- 희소 표현과는 다르게 단어 간의 유사도 를 계산으로 구할 수 있다는 장점.

# Embedding 레이어
- 단어의 분산 표현을 구현하기 위한 레이어.
-  컴퓨터가 알아서 n x k 형태의 분산 표현 사전을 만듦.
- 이것이 곧 Weight이고 파라미터입.
-  수많은 데이터를 통해 적합한 파라미터를 찾아가게 됨.
- Embedding 레이어는 입력으로 들어온 단어를 분산 표현으로 연결해 주는 역할을 하는데 그것이 Weight에서 특정 행을 읽어오는 것과 같아 이 레이어를 룩업 테이블(Lookup Table) 이라고 부르기도 함.
## - Embedding 사이즈를 정해주면  "단어를 더 깊게 표현해~" 라고도 말해줄 수 있음.
  - 그럼 Weight는 자연스럽게 단어의 개수, Embedding 사이즈로 정의됨

# 원-핫 인코딩

In [1]:
import tensorflow as tf

vocab = {      # 사용할 단어 사전 정의
    "i": 0,
    "need": 1,
    "some": 2,
    "more": 3,
    "coffee": 4,
    "cake": 5,
    "cat": 6,
    "dog": 7
}

sentence = "i i i i need some more coffee coffee coffee"
# 위 sentence
_input = [vocab[w] for w in sentence.split()]  # [0, 0, 0, 0, 1, 2, 3, 4, 4, 4]

vocab_size = len(vocab)   # 8

one_hot = tf.one_hot(_input, vocab_size)
print(one_hot.numpy())    # 원-핫 인코딩 벡터를 출력해 봅시다.

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [2]:
distribution_size = 2   # 보기 좋게 2차원으로 분산 표현하도록 하죠!
linear = tf.keras.layers.Dense(units=distribution_size, use_bias=False)
one_hot_linear = linear(one_hot)

print("Linear Weight")
print(linear.weights[0].numpy())

print("\nOne-Hot Linear Result")
print(one_hot_linear.numpy())

Linear Weight
[[ 0.2110765   0.30593204]
 [ 0.6935713  -0.19323993]
 [-0.11635119 -0.04150736]
 [ 0.711457    0.40476155]
 [ 0.266909    0.52930367]
 [ 0.22107226  0.34095275]
 [-0.46553138  0.16560155]
 [ 0.52522254  0.08885664]]

One-Hot Linear Result
[[ 0.2110765   0.30593204]
 [ 0.2110765   0.30593204]
 [ 0.2110765   0.30593204]
 [ 0.2110765   0.30593204]
 [ 0.6935713  -0.19323993]
 [-0.11635119 -0.04150736]
 [ 0.711457    0.40476155]
 [ 0.266909    0.52930367]
 [ 0.266909    0.52930367]
 [ 0.266909    0.52930367]]


In [3]:
#Tensorflow에서 Embedding 레이어를 선언
some_words = tf.constant([[3, 57, 35]])
# 3번 단어 / 57번 단어 / 35번 단어로 이루어진 한 문장입니다.

print("Embedding을 진행할 문장:", some_words.shape)
embedding_layer = tf.keras.layers.Embedding(input_dim=64, output_dim=100)
# 총 64개의 단어를 포함한 Embedding 레이어를 선언할 것이고,
# 각 단어는 100차원으로 분산표현 할 것입니다.

print("Embedding된 문장:", embedding_layer(some_words).shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

Embedding을 진행할 문장: (1, 3)
Embedding된 문장: (1, 3, 100)
Embedding Layer의 Weight 형태: (64, 100)


-  Embedding 레이어는 그저 단어를 대응 시켜 줄 뿐이니 미분이 불가능.
- 따라서 신경망 설계를 할 때, 어떤 연산 결과를 Embedding 레이어에 연결시키는 것은 불가능.

# RNN
- 순차 데이터를 처리하기 위해 고안된 것이 바로 Recurrent Neural Network 또는 Recurrent 레이어(이하 RNN).
- RNN의 입력으로 들어가는 모든 단어만큼 Weight를 만드는 게 아님.
- (입력의 차원, 출력의 차원)에 해당하는 단 하나의 Weight를 순차적으로 업데이트하는 것.
-  여러 번의 연산이 필요해 다른 레이어에 비해 느리다는 단점.
- 입력의 앞부분이 뒤로 갈수록 옅어져 손실이 발생 : 기울기 소실(vanishing Gradient)

In [4]:
sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

print("RNN에 입력할 문장:", sentence)

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

print("Embedding을 위해 단어 매핑:", sentence_tensor.numpy())
print("입력 문장 데이터 형태:", sentence_tensor.shape)

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("\nEmbedding 결과:", emb_out.shape)
print("Embedding Layer의 Weight 형태:", embedding_layer.weights[0].shape)

rnn_seq_layer = \
tf.keras.layers.SimpleRNN(units=64, return_sequences=True, use_bias=False)
rnn_seq_out = rnn_seq_layer(emb_out)

print("\nRNN 결과 (모든 Step Output):", rnn_seq_out.shape)
print("RNN Layer의 Weight 형태:", rnn_seq_layer.weights[0].shape)

rnn_fin_layer = tf.keras.layers.SimpleRNN(units=64, use_bias=False)
rnn_fin_out = rnn_fin_layer(emb_out)

print("\nRNN 결과 (최종 Step Output):", rnn_fin_out.shape)
print("RNN Layer의 Weight 형태:", rnn_fin_layer.weights[0].shape)

RNN에 입력할 문장: What time is it ?
Embedding을 위해 단어 매핑: [[2 3 0 1 4]]
입력 문장 데이터 형태: (1, 5)

Embedding 결과: (1, 5, 100)
Embedding Layer의 Weight 형태: (5, 100)

RNN 결과 (모든 Step Output): (1, 5, 64)
RNN Layer의 Weight 형태: (100, 64)

RNN 결과 (최종 Step Output): (1, 64)
RNN Layer의 Weight 형태: (100, 64)


In [5]:
lstm_seq_layer = tf.keras.layers.LSTM(units=64, return_sequences=True, use_bias=False)
lstm_seq_out = lstm_seq_layer(emb_out)

print("\nLSTM 결과 (모든 Step Output):", lstm_seq_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_seq_layer.weights[0].shape)

lstm_fin_layer = tf.keras.layers.LSTM(units=64, use_bias=False)
lstm_fin_out = lstm_fin_layer(emb_out)

print("\nLSTM 결과 (최종 Step Output):", lstm_fin_out.shape)
print("LSTM Layer의 Weight 형태:", lstm_fin_layer.weights[0].shape)


LSTM 결과 (모든 Step Output): (1, 5, 64)
LSTM Layer의 Weight 형태: (100, 256)

LSTM 결과 (최종 Step Output): (1, 64)
LSTM Layer의 Weight 형태: (100, 256)


# LSTM
- Long Short-Term Memory의 약어로 기울기 소실 문제를 해결하기 위해 고안된 RNN 레이어.
- 기본적인 바닐라(Simple) RNN보다 4배나 큰 Weight를 가지고 있음.
- 4배 깊은 RNN이라고 표현하기보다, 4종류의 서로 다른 Weight를 가진 RNN이라고 이해하시는 것이 좋음.
-  Cell state 라는 새로운 개념이 추가되는데, 긴 문장이 들어와도 이 Cell state 를 통해 오래된 기억 또한 큰 손실 없이 저장해 줌.
- Gate 들이 Cell state에 정보를 추가하거나 빼주는 역할.

# GRU
-  LSTM의 변형 모델.
- tf.keras.layers.GRU() 로 선언해서 사용.

# 양방향(Bidirectional) RNN
- 진행 방향에 변화를 준 RNN.
- 진행 방향이 반대인 RNN을 2개 겹쳐놓은 형태.
- 사용하고자 하는 레이어를 tf.keras.layers.Bidirectional() 로 감싸주면 됨.
-  주로 기계번역 같은 테스크에 유리.
- 번역기를 만들 때 양방향(Bidirectional) RNN 계열의 네트워크, 혹은 동일한 효과를 내는 Transformer 네트워크를 주로 사용하게 될 것.

In [6]:
import tensorflow as tf

sentence = "What time is it ?"
dic = {
    "is": 0,
    "it": 1,
    "What": 2,
    "time": 3,
    "?": 4
}

sentence_tensor = tf.constant([[dic[word] for word in sentence.split()]])

embedding_layer = tf.keras.layers.Embedding(input_dim=len(dic), output_dim=100)
emb_out = embedding_layer(sentence_tensor)

print("입력 문장 데이터 형태:", emb_out.shape)

bi_rnn = \
tf.keras.layers.Bidirectional(
    tf.keras.layers.SimpleRNN(units=64, use_bias=False, return_sequences=True)
)
bi_out = bi_rnn(emb_out)

print("Bidirectional RNN 결과 (최종 Step Output):", bi_out.shape)

입력 문장 데이터 형태: (1, 5, 100)
Bidirectional RNN 결과 (최종 Step Output): (1, 5, 128)


- Bidirectional RNN은 순방향 Weight와 역방향 Weight를 각각 정의하므로 우리가 앞에서 배운 RNN의 2배 크기 Weight가 정의.
- nits를 64로 정의해 줬고, 입력은 Embedding을 포함하여 (1, 5, 100), 그리고 양방향에 대한 Weight를 거쳐 나올 테니 출력은 (1, 5, 128) 